In [12]:
import pulp
import logging
import pandas as pd 

logging.getLogger().setLevel(level=logging.INFO)
logging.getLogger("yahoo_oauth").setLevel(level=logging.INFO)


from csh_fantasy_bot.bot import ManagerBot
from nhl_scraper.nhl import Scraper

from pulp import *

league_id = '403.l.41177'
week_number = 5
manager = ManagerBot(week_number, league_id=league_id)

my_scores = manager.my_team.scores()
# manager.score_comparer.print_week_results(my_scores)


In [22]:

# lets do second day of week, lots of guys can play
game_day = manager.week[0].to_pydatetime().date()
teams_playing_today = Scraper()._teams_playing_one_day(game_day)
my_roster = manager.my_team.roster().copy()
game_day_players = my_roster[my_roster.team_id.isin(teams_playing_today)]

game_day_players.head()
game_day_players = game_day_players[["eligible_positions", "name", "fpts"]]
game_day_players.head(20)

,eligible_positions,name,fpts
player_id,,,
3358,[D],Brent Burns,0.777493
4684,"[LW, RW]",Evander Kane,1.777869
5697,"[C, LW]",Tomas Hertl,0.981181
5986,[D],Darnell Nurse,1.098989
5985,[C],Sean Monahan,1.019299
6083,"[C, LW]",Andrew Copp,0.887706
5370,[D],Dougie Hamilton,1.159915
5431,[C],Vincent Trocheck,1.239122
7902,[LW],Brady Tkachuk,1.506455


In [23]:
opponent_scores_for_day = manager.opponent.scores()
opponent_scores_for_day.head()

# opponent_scores_for_day[opponent_scores_for_day.play_date == game_day].head()

G    A  +/-  PIM  SOG   FW  HIT score_type
play_date  player_id                                              
2021-02-15 5096       0.0  0.0 -1.0  4.0  5.0  0.0  3.0          a
           6744       0.0  0.0 -1.0  0.0  2.0  8.0  0.0          a
           5691       NaN  NaN  NaN  NaN  NaN  NaN  NaN          a
           5928       1.0  0.0  2.0  0.0  2.0  2.0  0.0          a
           3641       1.0  1.0 -1.0  0.0  1.0  0.0  2.0          a

In [24]:
# availables.eligible_positions.map(set(['LW']).issubset)

def score(goals, assists):
    opponent_goals = 6.8
    oppoonent_assists = 12

    
prob = pulp.LpProblem('NHL', LpMaximize)

player_vars = {player_id:pulp.LpVariable(str(player_id), cat='Binary') for player_id in game_day_players.index.values}

#adds objective function
prob += (lpSum(game_day_players.loc[p_id, 'fpts']*player_vars[p_id] for p_id in game_day_players.index.values))

# prob += (lpSum(game_day_players.positions.loc[p_id,'C']*player_vars[p_id] for p_id in game_day_players.index.values) == 2)


AttributeError: 'DataFrame' object has no attribute 'positions'

In [21]:


# d = {i:[] for i in pd.Series(game_day_players['eligible_positions']).unique()}


# pos_num_available = {
#     "C": 2,
#     "LW": 2,
#     "RW": 2,
#     "D": 4,
# }

# for pos in pos_num_available.keys():
#     available_pos = availables[availables.eligible_positions == pos]
#     salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
#     point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
#     salaries[pos] = salary
#     points[pos] = point
